In [ ]:
# Importing modules from the commutazzio library and standard Python libraries
from commutazzio.random import RandomFiltrationPointCloudModel
from commutazzio.filtration import CLFiltrationDB
from commutazzio.plot import ComplementaryTrianglesPlot as Visualizer1
import random
import numpy as np
from commutazzio.utils import filepath_generator
import pandas as pd
from io import StringIO
import yaml
# Loading model parameters from YAML configuration file
with open('./model_parameters.yaml', 'r') as file:
    model_parameters = yaml.safe_load(file)
POINT_CLOUD_PARAMS = model_parameters['point_cloud']
# Setting up database name and directory
DB_NAME = 'PointCloudModel_tutorial'
DIR_NAME = 'tutorial_databases'
# Creating the tutorial databases directory if it doesn't exist
import os
if not os.path.exists(DIR_NAME):
    os.makedirs(DIR_NAME)

In [ ]:
# Function to generate and analyze filtrations for a specified number of Point Cloud Model instances
def generate_filtration_point_cloud_model(num_instances=5):
    create_new_db=True
    # Initialize databases for CL(4) and CL(n) filtrations
    db4 = CLFiltrationDB(filepath_generator(dirname=DIR_NAME,filename=f"{DB_NAME}_4", extension='db',overwrite=not create_new_db),create_new_db=create_new_db)
    dbn = CLFiltrationDB(filepath_generator(dirname=DIR_NAME,filename=f"{DB_NAME}_n", extension='db',overwrite=not create_new_db),create_new_db=create_new_db)
    # Extract parameters from configuration file
    ladder_len_min = POINT_CLOUD_PARAMS['ladder_len_min']
    ladder_len_max = POINT_CLOUD_PARAMS['ladder_len_max']
    pts_min = POINT_CLOUD_PARAMS['pts_min']
    pts_max = POINT_CLOUD_PARAMS['pts_max']
    enable_multi_processing = POINT_CLOUD_PARAMS['enable_multi_processing']
    num_cores = POINT_CLOUD_PARAMS['num_cores']
    verbose = POINT_CLOUD_PARAMS['verbose']
    i=0
    while i<num_instances: # Loop through the specified number of instances
        num_pts=random.randint(pts_min,pts_max) # Determine the number of points for this model
        # Initialize a random filtration
        result=RandomFiltrationPointCloudModel(num_pts=num_pts,ladder_length_min=ladder_len_min,ladder_length_max=ladder_len_max,enable_multi_processing=enable_multi_processing,num_cores=num_cores,verbose=verbose)
        i+=1
        print(i)
        if filtration:=result.output["CL(4)"]:
            decomp = filtration.info['decomp']
            # Identify non-intervals in the decomposition
            non_intervals = {k:v for k,v in decomp.items() if k[0] == 'N'}
            if non_intervals:
                print(f'non_intervals found @ {i}',flush=True)
            db4.add_filtration(filtration)
        if filtration:=result.output["CL(n)"]:
            non_intervals=(pd.read_csv(StringIO(filtration.info['lines']),index_col=0).multiplicity<0).any()
            dbn.add_filtration(filtration) # Add CL(n) filtration results to the database

In [ ]:
# generate five filtrations and store them in the database
generate_filtration_point_cloud_model(5)

In [ ]:
# Import necessary functions from the db_conversion_utils module within the commutazzio.random package
from commutazzio.random.db_conversion_utils import sqlite_to_df_cl4_pc, sqlite_to_df_cln_pc, collect_data_from_db_files

# Specify the directory containing the database files for processing
dirnames = ["./tutorial_databases/"]

# Define the filename patterns for identifying the specific database files to process
fn_pattern_cl4 = "PointCloudModel_tutorial_4" 
fn_pattern_cln = "PointCloudModel_tutorial_n"

# Use the collect_data_from_db_files function to search within the specified directory (or directories)
# for database files matching the given filename patterns. Each matching file's data is then processed
# by the respective conversion function (sqlite_to_df_cl4_pc or sqlite_to_df_cln_pc) to create a DataFrame
# containing structured data extracted from the databases.
df_cl4_pcm = collect_data_from_db_files(dirnames, fn_pattern_cl4, sqlite_to_df_cl4_pc)  # Collect and convert CL4 data
df_cln_pcm = collect_data_from_db_files(dirnames, fn_pattern_cln, sqlite_to_df_cln_pc)  # Collect and convert CLn data



